# TMA4320 Biophysics project
### Oliver Ruden, Åsmund Mjøs & Astrid Mysterud

Polymers are the building blocks of DNA, RNA and proteins. Polymers themselves consist of repeating units called monomers. In this project, polymers will be represented numerically in order to make energy calculations as the polymers rotate around their monomers. FORTSETTELSE INNTRODUKSJON.

(Kommentere på at følgende setning svarer på 1a?) A polymer with $N$ monomers will be represented by a $N\times 2$-array containing the $N$ sets of $x$- and $y$-coordinates of their monomers. The polymer is visualized in an $N\times N$-grid with the origin placed in the bottom left corner.

The following function $\texttt{createPolymer}$, as described in task 1b), recieves the argument $N$ and returns a polymer of size $N$ shaped as a horizontal line. To determine the $y$-coordinate, integer division is used. 

In [1]:
# First, we import necessary libaries
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [3]:
def createPolymer(N):
    polymer = np.zeros((N, 2))                             # initialize an Nx2-array to represent the polymer
    polymer[:, 1] = N // 2                                 # place same y-coordinate for all monomers
    polymer[:, 0] = np.array([x for x in range(N)])        # place x-coordinat
    return polymer                                         # returns polymer shaped as a horizontal line

To answer task **1c)** In order to follow the rules that determines valid polymers, the energy will remain unchanged after a rotation around an endpoint. The energy of a polymer is determined by *noe* between its monomers. When a polymers is rotated around an endpoint, its monomers does not recieve any new neighbours and as a result the energy of the polymer does not change. 

In algorithm 1 and 2 from the project description, we are to choose a random monomer to rotate around. To decrease the run time of the code, we've decided not to include the edge monomers, since this does not change the *noe* of the monomer.

The following code implements a function that visualizes the polymer, as asked for in task **1d)**. The polymer is shown in the yellow/green squares. In the $N\times N$-grid *hvor polymeret er*, we change the background value to $-N/2$ to *framheve* our polymer (to be able to uniquely identify monomers). Monomers with small indices are represented by *en farge*, while monomers with greater indices are represented by *en annen farge*. We illustrate it in a $(N+1)\times (N+1)$-grid, because *denne grunnen*. 

In [4]:
def illustratePolymer(polymer):
    N = len(polymer)                               # calculate the size of the input polymer
    grid = np.zeros((N + 1, N + 1))                # initialize an (N+1) * (N+1) grid
    grid -= int(N/2)                               # set background value to -N / 2 for a better visualization of monomers with small indices

    for monomerNumber in range(N):                 # loop through all monomers of the input polymer
        x = int(polymer[monomerNumber, 0])         # collect x-coordinate of the monomer
        y = int(polymer[monomerNumber, 1])         # collect y-coordinate of the monomer
        grid[y, x] = monomerNumber + 1             # set the value of the point (x, y) in the grid to the index of the monomer + 1 (i.e. start at 1)

    plt.pcolormesh(grid)
    plt.show()

While rotating a polymer around its monomers, one easily ends up with a polymer that does not meet the requirements as listed in task **1e)**. The following function receives a polymer and its size $N$, and checks whether it is valid or not. The function $\texttt{validPolymer}$ returns $\texttt{True}$ if the polymer is valid, and $\texttt{False}$ otherwise. To make sure that no monomers have the same coordinates, we initialize the set $\texttt{coordinateSet}$, where we add the monomer-coordinates as we loop through the monomers of the polymer. Once a monomer with already taken coordinates is encountered, $\texttt{validPolymer}$ returns $\texttt{False}$. If a monomer has unique coordinates $(x, y)$, the tuple $(x, y)$ is added to $\texttt{coordinateSet}$.

In [5]:
def validPolymer(polymer, N):
    if len(polymer) != N:                                      # check if the length of the polymer does not match the input length
        return False
    
    coordinateSet = set()                                      # initialize empty set
    coordinateSet.add((polymer[0, 0], polymer[0, 1]))          # add x- and y-coordinate of the first monomer

    for monomerNumber in range(1, N):                          # loop through the remaining monomers of the polymer

        if (polymer[monomerNumber, 0], polymer[monomerNumber, 1]) in coordinateSet: # check if the monomers coordinates are taken by a preceding monomer
            return False                                                            
        else: 
            coordinateSet.add((polymer[monomerNumber, 0], polymer[monomerNumber, 1])) # if the coordinates are unique, add them to coordinateSet

        xDiff = np.abs(polymer[monomerNumber, 0] - polymer[monomerNumber - 1, 0])     # x-distance to the preceding monomer
        yDiff = np.abs(polymer[monomerNumber, 1] - polymer[monomerNumber - 1, 1])     # y-distance to the preceding monomer
        if xDiff + yDiff != 1:    # if the sum of the x- and y-distance to the preceding monomer is not 1, the preceding monomer is not its neighbour
            return False          
        
    return True    

Next code block implements the function described in task **1f)**. The function $\texttt{rotatePolymer}$ receives an input polymer and a monomer which the polymer will rotate around. $\texttt{rotatePolymer}$ also receives a boolean $\texttt{positiveDirection}$. $\texttt{positiveDirection=True}$ rotates the polymer in the positive direction, while $\texttt{positiveDirection=False}$ rotates the polymer in the negative direction. 

The placement of each monomer in the polymer after a rotation around a monomer $m_{rot}$, is determined by it's relative distance in $x$-direction $\Delta x$ to $m_{rot}$ and it's relative distance in $y$-direction $\Delta y$ to $m_{rot}$. We look at the monomers positions relative to $m_{rot}$ as free vectors, and since we will either be rotating them in positive or negative direction, we can simply multiply by the rotation matrix, either positive direction, or negative direction.

In positive rotation, the original vector $(1,0)$ will go to $(0,1)$ and the vector $(0,1)$ will go to $(0,-1)$. In negative rotation, the original vector $(1,0)$ will go to $(0,-1)$ and $(0,1)$ will go to $(1,0)$ in negative rotation. This is essentially all the information we need to implement rotation, since rotation is a linear operation, and these two vectors span out the plane.


In [6]:
def rotatePolymer(polymer, monomer, positiveDirection):
    monomer -= 1                                            # subtract 1 from the monomerNumber, as the polymer-array is zero-indexed
    middleMonomer = len(polymer) // 2                       # find monomerNumber of the middle monomer in the polymer
    x, y = polymer[monomer]                                 # collect x- and y-coordinates of the monomer the polymer will rotate around
    newPolymer = np.zeros((len(polymer),2))                 # initialize a new polymer, because working inplace changed *noe*

    if middleMonomer > monomer:
        newPolymer[monomer:] = polymer[monomer:]
        newPolymer[:monomer, 0] = (2 * positiveDirection - 1) * (polymer[:monomer, 1] - y) + x
        newPolymer[:monomer, 1] = (1 - 2 * positiveDirection) * (polymer[:monomer, 0] - x) + y
        """
        Positive direction:
        delta x = delta y
        delta y = - delta x
        Negative direction:
        delta x = - delta y
        delta y = delta x

        Also uses that True is represented as 1 and False as 0.
        """
        return newPolymer
    newPolymer[:monomer+1] = polymer[:monomer+1]
    newPolymer[monomer+1:,0] = (1-2*positiveDirection)*(polymer[monomer+1:,1]-y)+x
    newPolymer[monomer+1:,1] = (2*positiveDirection-1)*(polymer[monomer+1:,0]-x)+y
    """
        Positive direction:
        delta y = delta x
        delta x = - delta y
        Negative direction:
        delta y = - delta x
        delta x = delta y
    """
    return newPolymer

## g)
Her roterer vi masse

In [16]:
@jit
def rotateNTimes(N, Ns):
    rotationsMade = 0
    polymer = createPolymer(N)

    for i in range(Ns):
        monomer = np.random.randint(2, N)
        positivRetning = np.random.choice([True, False])

        twistedPolymer = rotatePolymer(polymer, monomer, positivRetning)
        if validPolymer(twistedPolymer, N):
            rotationsMade += 1
            polymer = twistedPolymer

    return polymer, rotationsMade

# h)

In [2]:
# kode!

# i) 

In [1]:
# kode til i!

# j) 

In [3]:
# oppgave j!

# 2a)

In [4]:
# kode!

# 2b)

In [5]:
# kode!

# 2c)

In [6]:
# kode!

# 2d)

In [7]:
# kode!

# 2e)

In [8]:
# kode!

# 2f)

In [9]:
# kode!

# 2g)

In [10]:
# kode!

# 2h)

In [11]:
# kode!